## **Summary**

This Notebook implements an integer program solution to scorito Top Coach team selection halfway Season 2021/2022 to optimize the number of obtained points so far given budgetary and other costraints. Data is extracted from Scorito API's and the integer program is solved using Pulp package. Limitations considered are budgetary constraints, max players per team and max players per positions. Bench and grandstand scoring features have not been included but can be easily implemented.

Notebook is structured as follows:
- Relevant installs, imports and settings are installed in the first part.
- Data is extracted from Scorito API's. You can find the API's in the network tab at the topcoach website. An API's is extracted for the players scores per week and the results are summed over all the weeks. The total score is then calculated by summing over all the different score categories. This dataset is merged with the pricelist of players. The merge is not one-to-one because some players have left and some have joined the club.
- Relevant data for integer porgramming are sent to dictionairies to allow easy access from pulp. The objective function and the constraints are formalized. The optimal team is the result.

Optimal Team:
- R. Pasveer
- M. Branderhorst

- D. Blind
- J. Timber
- N. Mazraoui
- R. Pröpper
- T. Malacia

- B. Ramselaar
- J. Bruijn
- R. Maulun
- S. Bakari
- S. Tronstad
- T. Halilovic

- D. Limnios 
- I. Kallon
- J. Odgaard
- R. Oratmangoen
- M. Seuntjens


## **Installs, Imports and Settings**

In [163]:
!pip install pulp &> /dev/nul

In [167]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import requests
import pulp
from pulp import *

In [165]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 50)
pd.options.display.float_format = '{:,.4f}'.format
pd.options.display.max_colwidth = 100 #To see all feature names completely

In [166]:
# Change default plot size
plt.rcParams['figure.figsize'] = (15, 7)

# Set display settings
rc = {'lines.linewidth': 2, 'font.sans-serif': ['DejaVu Sans']}
sns.set_context('notebook', rc=rc, font_scale=1.25)
sns.set_style('whitegrid', rc=rc)

## **Extraction Data and Preprocessing Data**

In [81]:
# Look for network tab at https://www.scorito.com/footballmanager/180 to find API's

#Get player points (unfortunately only available per round, so we sum the rounds manually: round 1 = 3283, round 18 = 3300)
player_scores = pd.DataFrame()
for i in range(3283,3301):
  URL = "https://football.scorito.com/footballmanager/v1.0/playerpoints/marketround/"+str(i)
  page = requests.get(URL)
  result = pd.json_normalize(page.json(), "Content")
  player_scores = pd.concat([player_scores, result]).groupby(['PlayerId']).sum().reset_index()

# Get player costs
URL = "https://football.scorito.com/footballmanager/v1.0/players/marketround/3301" 
page = requests.get(URL)
player_costs = pd.json_normalize(page.json(), "Content")


player_scores['Total_score'] = player_scores[['PointsAssist','PointsDraw','PointsGoal','PointsHattrick','PointsKeptZero','PointsMissedpenalty',
                                              'PointsOwngoal','PointsPlaytime','PointsRedcarddirect','PointsSavedpenalty','PointsTwogoalsagainst',
                                              'PointsWins','PointsYellowcard']].sum(axis=1)

#Merge costs and scores:
data = player_costs.merge(player_scores, how='left', on='PlayerId') # Some players do not have cost cause they have been sold

In [78]:
# Data is aligned with Scorito Scores (see Scorito)
data.sort_values(by='Total_score',ascending=False).head(5)

,FullName,LastName,NameShort,PlayerId,Position,Price,TeamId,AvailabilityStatus,ValueChangeType,PointsAssist,PointsDraw,PointsGoal,PointsHattrick,PointsKeptZero,PointsMissedpenalty,PointsOwngoal,PointsPlaytime,PointsRedcarddirect,PointsSavedpenalty,PointsTwogoalsagainst,PointsWins,PointsYellowcard,Total_score
94,Noussair Mazraoui,Mazraoui,N. Mazraoui,19852,2,5000000,1,0,1,96.0000,12.0000,384.0000,0.0000,576.0000,0.0000,0.0000,504.0000,0.0000,0.0000,0.0000,198.0000,-24.0000,"1,746.0000"
98,Jurrien Timber,Timber,J. Timber,26422,2,4500000,1,0,1,48.0000,18.0000,192.0000,0.0000,672.0000,0.0000,0.0000,612.0000,0.0000,0.0000,-24.0000,216.0000,0.0000,"1,734.0000"
198,Guus Til,Til,G. Til,13902,4,6000000,2,0,1,80.0000,18.0000,800.0000,0.0000,216.0000,0.0000,0.0000,504.0000,0.0000,0.0000,-72.0000,216.0000,-48.0000,"1,714.0000"
91,Daley Blind,Blind,D. Blind,332,2,4500000,1,0,1,96.0000,18.0000,96.0000,0.0000,672.0000,0.0000,0.0000,576.0000,0.0000,0.0000,-24.0000,216.0000,0.0000,"1,650.0000"
202,Bryan Linssen,Linssen,B. Linssen,309,8,5000000,2,0,1,160.0000,18.0000,768.0000,0.0000,0.0000,0.0000,0.0000,476.0000,0.0000,0.0000,0.0000,198.0000,-48.0000,"1,572.0000"


In [87]:
# Some players do not have results for every game so we need to replace missing data by 0
data.fillna(0, inplace=True)

# Map position and club data from ID:
clubmapping = {10: 'FC Twente', 3: 'FC Groningen',21:'Go Ahead Eagles',2 : 'Feyenoord',22:'SC Cambuur',6: 'NEC', 20: 'PEC Zwolle', 692:'Fortuna Sittard',11:'FC Utrecht',
               4:'SC Heerenveen', 7:'PSV', 12:'Vitesse',8:'RKC Waalwijk',351:'Sparta Rotterdam',14:'Heracles Almelo', 19:'Willem II', 13: 'AZ', 1:'Ajax'}

positionmapping = {1:'Keeper', 2:'Defender', 4:'Midfielder',8:'Forwarder'}

data['Team'] = data['TeamId'].map(clubmapping)
data['Position_name'] = data['Position'].map(positionmapping)

## **Integer Programming**

We formulate the objective to maximize the sum of the number of points have received so far, not taking into account scoring for subs and. 

Integer linear program: <br>
$$\begin{eqnarray}
\text{max } && \sum_{j\in J}  p_j x_{j}\\
\text{subject to } && \sum_{j\in J} c_{j}x_{j} \leq 48.000.000 && (1) \\
&& \sum_{t\in T} x_{t} \leq 4 & \forall \text{ Team }  T & (2) \\
&& \sum_{p\in P} x_{p} \leq b_p & \forall \text{ Position }  P & (3) \\
\end{eqnarray}$$

Here,
- (1) The budgetary constraint.
- (2) The maximum players per team constraint.
- (3) The maximum players per position ($b_p$) constraint where 
    - $b_{keeper}=2$
    - $b_{defender}=5$  
    - $b_{midfielder}=6$
    - $b_{forwarder}=5$.

In [155]:
# Put the data needed for integer programming in dictionaries. Use player ID as index.
points = dict(zip(data.NameShort, data.Total_score))
prices = dict(zip(data.NameShort, data.Price))

# Select Players per Team and position
players_per_team = data.groupby('TeamId')['NameShort'].apply(list).reset_index()
position_per_team = data.groupby('Position')['NameShort'].apply(list).reset_index()

teams = dict(zip(players_per_team.TeamId, players_per_team.NameShort)) 
position = dict(zip(position_per_team.Position, position_per_team.NameShort))

# Put the data needed for integer programming in dictionaries. Use player ID as index.
players = data.PlayerId.to_list()
player_names = data.NameShort.to_list()

In [161]:
# Creates the boundless Variables as Integers
vars = LpVariable.dicts('', player_names, 0, 1, LpInteger)

# Creates the 'prob' variable to contain the problem data
prob = LpProblem("Maximum Player Points Problem", LpMaximize)

# Creates the objective function
prob += lpSum([vars[a] * points[a] for a in player_names]), "Maximize the total points"

# Set constraints on available money
prob += lpSum([vars[a] * prices[a] for a in player_names]) <= 48000000 

# Set maximum constraint of 4 players per team
for i in data.TeamId.unique():
  prob += lpSum([vars[a] for a in teams[i]]) <= 4 

# Select 2 keepers, 5 defenders, 6 midfielders, 5 forwarders (Is also constraint on number of total players)
prob += lpSum([vars[a] for a in position[1]]) == 2
prob += lpSum([vars[a] for a in position[2]]) == 5
prob += lpSum([vars[a] for a in position[4]]) == 6
prob += lpSum([vars[a] for a in position[8]]) == 5


# Solve ILP via Gurobi
prob.solve()

/usr/local/lib/python3.7/dist-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [162]:
for variable in prob.variables():
    if variable.varValue > 0:
      print ("{} = {}".format(variable.name, variable.varValue))
print("Total Points = ", value(prob.objective))
print(LpStatus[prob.status])

_B._Ramselaar = 1.0
_D._Blind = 1.0
_D._Limnios = 1.0
_I._Kallon = 1.0
_J._Bruijn = 1.0
_J._Odgaard = 1.0
_J._Timber = 1.0
_M._Branderhorst = 1.0
_M._Seuntjens = 1.0
_N._Mazraoui = 1.0
_R._Maulun = 1.0
_R._Oratmangoen = 1.0
_R._Pasveer = 1.0
_R._Pröpper = 1.0
_S._Bakari = 1.0
_S._Tronstad = 1.0
_T._Halilovic = 1.0
_T._Malacia = 1.0
Total Points =  20072.0
Optimal
